# RNN을 이용한 텍스트 생성
- '경마장에 있는 말이 뛰고 있다'
- '그의 말이 법이다'
- '가는 말이 고와야 오는 말이 곱다'

## 1. 데이터 전처리

In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
# 3개의 문장을 변수에 저장
text="""경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [3]:
# 단어 집합 생성
t = Tokenizer()
t.fit_on_texts([text])

In [4]:
t.word_index

{'말이': 1,
 '경마장에': 2,
 '있는': 3,
 '뛰고': 4,
 '있다': 5,
 '그의': 6,
 '법이다': 7,
 '가는': 8,
 '고와야': 9,
 '오는': 10,
 '곱다': 11}

In [5]:
# 단어 집합 크기 설정
vocab_size = len(t.word_index) + 1
# 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만,
# 케라스 원-핫 인코딩에서 배열의 인덱스가 0부터 시작하기 때문에
# 배열의 크기를 실제 단어 집합의 크기보다 +1로 생성해야함 
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


In [6]:
sequences = []
for line in text.split('\n'): # \n을 기준으로 문장 토큰화
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

학습에 사용할 샘플의 개수: 11


In [7]:
sequences

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6, 1],
 [6, 1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

In [8]:
# 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
max_len=max(len(s) for s in sequences) 
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 6


In [9]:
# 전체 샘플의 길이를 6(가장 긴 샘플의 길이)으로 패딩
# 'pre' 옵션을 주면 앞을 0으로 패딩
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

In [10]:
sequences[:5]

array([[0, 0, 0, 0, 2, 3],
       [0, 0, 0, 2, 3, 1],
       [0, 0, 2, 3, 1, 4],
       [0, 2, 3, 1, 4, 5],
       [0, 0, 0, 0, 6, 1]])

In [11]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]
# 리스트의 마지막 값을 제외하고 저장한 것은 X
# 리스트의 마지막 값만 저장한 것은 y. 이는 레이블에 해당됨.

In [12]:
X[:5]

array([[0, 0, 0, 0, 2],
       [0, 0, 0, 2, 3],
       [0, 0, 2, 3, 1],
       [0, 2, 3, 1, 4],
       [0, 0, 0, 0, 6]])

In [13]:
# 레이블 데이터 y에 대해서 원-핫 인코딩을 수행
y = to_categorical(y, num_classes=vocab_size)

In [14]:
y

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

## 2. 모델 설계 및 학습

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [25]:
embedding = Embedding(vocab_size, 10, input_length=max_len-1) # 레이블을 분리하였으므로 X의 길이는 5
rnn = SimpleRNN(32)
output = Dense(vocab_size, activation='softmax')

In [26]:
# 임베딩 벡터는 10차원, 은닉 상태 크기는 32

model = Sequential()
model.add(embedding)
model.add(rnn)
model.add(output)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 5, 10)             120       
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                1376      
_________________________________________________________________
dense_1 (Dense)              (None, 12)                396       
Total params: 1,892
Trainable params: 1,892
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [28]:
model.fit(X, y, epochs=200, verbose=2)

Train on 11 samples
Epoch 1/200
11/11 - 1s - loss: 2.4598 - accuracy: 0.0909
Epoch 2/200
11/11 - 0s - loss: 2.4455 - accuracy: 0.0909
Epoch 3/200
11/11 - 0s - loss: 2.4310 - accuracy: 0.1818
Epoch 4/200
11/11 - 0s - loss: 2.4162 - accuracy: 0.2727
Epoch 5/200
11/11 - 0s - loss: 2.4009 - accuracy: 0.3636
Epoch 6/200
11/11 - 0s - loss: 2.3852 - accuracy: 0.3636
Epoch 7/200
11/11 - 0s - loss: 2.3689 - accuracy: 0.4545
Epoch 8/200
11/11 - 0s - loss: 2.3521 - accuracy: 0.3636
Epoch 9/200
11/11 - 0s - loss: 2.3345 - accuracy: 0.3636
Epoch 10/200
11/11 - 0s - loss: 2.3162 - accuracy: 0.3636
Epoch 11/200
11/11 - 0s - loss: 2.2971 - accuracy: 0.3636
Epoch 12/200
11/11 - 0s - loss: 2.2771 - accuracy: 0.3636
Epoch 13/200
11/11 - 0s - loss: 2.2562 - accuracy: 0.3636
Epoch 14/200
11/11 - 0s - loss: 2.2344 - accuracy: 0.3636
Epoch 15/200
11/11 - 0s - loss: 2.2118 - accuracy: 0.3636
Epoch 16/200
11/11 - 0s - loss: 2.1882 - accuracy: 0.3636
Epoch 17/200
11/11 - 0s - loss: 2.1639 - accuracy: 0.3636
Epo

Epoch 142/200
11/11 - 0s - loss: 0.4706 - accuracy: 0.9091
Epoch 143/200
11/11 - 0s - loss: 0.4631 - accuracy: 0.9091
Epoch 144/200
11/11 - 0s - loss: 0.4557 - accuracy: 0.9091
Epoch 145/200
11/11 - 0s - loss: 0.4484 - accuracy: 0.9091
Epoch 146/200
11/11 - 0s - loss: 0.4412 - accuracy: 0.9091
Epoch 147/200
11/11 - 0s - loss: 0.4341 - accuracy: 0.9091
Epoch 148/200
11/11 - 0s - loss: 0.4271 - accuracy: 0.9091
Epoch 149/200
11/11 - 0s - loss: 0.4202 - accuracy: 0.9091
Epoch 150/200
11/11 - 0s - loss: 0.4134 - accuracy: 0.9091
Epoch 151/200
11/11 - 0s - loss: 0.4067 - accuracy: 0.9091
Epoch 152/200
11/11 - 0s - loss: 0.4001 - accuracy: 0.9091
Epoch 153/200
11/11 - 0s - loss: 0.3936 - accuracy: 0.9091
Epoch 154/200
11/11 - 0s - loss: 0.3871 - accuracy: 0.9091
Epoch 155/200
11/11 - 0s - loss: 0.3808 - accuracy: 0.9091
Epoch 156/200
11/11 - 0s - loss: 0.3746 - accuracy: 0.9091
Epoch 157/200
11/11 - 0s - loss: 0.3684 - accuracy: 0.9091
Epoch 158/200
11/11 - 0s - loss: 0.3623 - accuracy: 0.90

## 3. 모델 검증

In [19]:
# 모델 검증용 문장을 생성하는 함수
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=5, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
        
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [20]:
# '경마장에' 라는 단어 뒤에는 총 4개의 단어가 있으므로 3번 예측
print(sentence_generation(model, t, '경마장에', 3))

경마장에 있는 말이 뛰고


In [22]:
print(sentence_generation(model, t, '그의', 2)) # 2번 예측

그의 말이 법이다


In [23]:
print(sentence_generation(model, t, '가는', 5)) # 5번 예측

가는 말이 고와야 오는 말이 곱다


In [24]:
print(sentence_generation(model, t, '가는', 3)) # 3번 예측

가는 말이 고와야 오는


### Weights and bias 확인

In [29]:
embedding.get_config()

{'name': 'embedding_1',
 'trainable': True,
 'batch_input_shape': (None, 5),
 'dtype': 'float32',
 'input_dim': 12,
 'output_dim': 10,
 'embeddings_initializer': {'class_name': 'RandomUniform',
  'config': {'minval': -0.05, 'maxval': 0.05, 'seed': None}},
 'embeddings_regularizer': None,
 'activity_regularizer': None,
 'embeddings_constraint': None,
 'mask_zero': False,
 'input_length': 5}

In [30]:
rnn.get_config()

{'name': 'simple_rnn_1',
 'trainable': True,
 'dtype': 'float32',
 'return_sequences': False,
 'return_state': False,
 'go_backwards': False,
 'stateful': False,
 'unroll': False,
 'time_major': False,
 'units': 32,
 'activation': 'tanh',
 'use_bias': True,
 'kernel_initializer': {'class_name': 'GlorotUniform',
  'config': {'seed': None}},
 'recurrent_initializer': {'class_name': 'Orthogonal',
  'config': {'gain': 1.0, 'seed': None}},
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'kernel_regularizer': None,
 'recurrent_regularizer': None,
 'bias_regularizer': None,
 'activity_regularizer': None,
 'kernel_constraint': None,
 'recurrent_constraint': None,
 'bias_constraint': None,
 'dropout': 0.0,
 'recurrent_dropout': 0.0}

In [31]:
output.get_config()

{'name': 'dense_1',
 'trainable': True,
 'dtype': 'float32',
 'units': 12,
 'activation': 'softmax',
 'use_bias': True,
 'kernel_initializer': {'class_name': 'GlorotUniform',
  'config': {'seed': None}},
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'kernel_regularizer': None,
 'bias_regularizer': None,
 'activity_regularizer': None,
 'kernel_constraint': None,
 'bias_constraint': None}

In [37]:
emb_weights = embedding.get_weights()
len(emb_weights)

1

In [38]:
emb_weights[0].shape

(12, 10)

In [39]:
emb_weights[0]

array([[-1.24945924e-01,  1.18594788e-01,  1.40483916e-01,
        -1.01346977e-01, -6.83053136e-02, -1.11641340e-01,
         3.11846063e-02,  2.92216353e-02, -6.84486702e-03,
        -6.19681813e-02],
       [-2.52110213e-02, -1.67641297e-01,  6.53932244e-02,
         1.61819488e-01, -1.41929343e-01,  1.31399453e-01,
         1.92326367e-01,  1.93178371e-01, -2.06788331e-01,
         1.66768417e-01],
       [ 9.59129706e-02, -8.74571204e-02,  1.46076441e-01,
         2.17405528e-01,  1.93239838e-01,  9.30028856e-02,
        -2.23317966e-01,  1.51342675e-01, -1.91816822e-01,
        -7.48132914e-02],
       [ 9.40247476e-02, -1.68502927e-01,  1.70027032e-01,
         8.05505514e-02,  1.99057385e-01,  1.66174471e-01,
         1.12766884e-01,  1.37891278e-01,  2.36875415e-01,
         1.40911728e-01],
       [ 1.43615052e-01,  1.05324984e-01, -6.99244961e-02,
        -1.58700421e-01,  1.29248589e-01,  1.35837570e-01,
        -1.26958519e-01,  1.01560973e-01,  5.00163175e-02,
        -1.

In [32]:
rnn_weights = rnn.get_weights()
type(rnn_weights)

list

In [33]:
len(rnn_weights)

3

In [34]:
rnn_weights[0].shape, rnn_weights[1].shape, rnn_weights[2].shape

((10, 32), (32, 32), (32,))

In [35]:
wx = rnn_weights[0]
wh = rnn_weights[1]
bias = rnn_weights[2]

In [36]:
bias

array([-0.00471539, -0.02431854,  0.01640963, -0.00084666,  0.00050329,
       -0.10412057,  0.04895264,  0.02990082,  0.05133681,  0.01698031,
        0.11403184,  0.07377192, -0.02001483,  0.03058521,  0.13110825,
        0.00192891, -0.01367764, -0.01876871, -0.01357221,  0.01342565,
       -0.02761317,  0.02883648, -0.02429904,  0.0424956 ,  0.05923627,
       -0.0691328 , -0.06474349, -0.06739316, -0.12345801,  0.01852162,
        0.00937177,  0.04982855], dtype=float32)